In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as tf
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm
import PIL

### Eval

###  Prepare data

In [4]:
CROP_SIZE = 128
class TrainDataset(Dataset):
    def __init__(self, dest, path):
        super(TrainDataset, self).__init__()
        self.image_names = [os.path.join(x,path) for x in os.listdir(path)]
        self.lr_preprocess = tf.Compose([tf.ToPILImage(),tf.Resize((CROP_SIZE,CROP_SIZE), interpolation = PIL.Image.BICUBIC),tf.ToTensor()])
        self.hr_preprocess = tf.Compose([tf.ToTensor()])
        self.len = len(self.image_names)
    def __len__(self):
        return self.len
    def __getitem__(self, index):
        img = self.image_names[index]
        hr = self.hr_preprocess(PIL.Image.open(img))
        lr = self.lr_preprocess(hr)
        return lr,hr

In [3]:
lr_preprocess = tf.Compose([tf.ToPILImage(),tf.Resize((CROP_SIZE,CROP_SIZE), interpolation = PIL.Image.BICUBIC),tf.ToTensor()])
hr_preprocess = tf.Compose([tf.ToTensor()])
img =PIL.Image.open(path)
img_hr = hr_preprocess(PIL.Image.open(path))
img_lr = lr_preprocess(img_hr)

NameError: name 'path' is not defined

# Model

<img src = "https://miro.medium.com/max/4916/1*zsiBj3IL4ALeLgsCeQ3lyA.png">

### Generator Network

In [5]:
#Residual Block
class Residual_Block(nn.Module):
    def __init__(self):
        super(Residual_Block, self).__init__()
        self.net = nn.Sequential(
        nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size = 3, padding = 1),
        nn.BatchNorm2d(num_features = 64),
        nn.PReLU(),
        nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size = 3, padding = 1),
        nn.BatchNorm2d(num_features = 64)
        )
    def forward(self, x):
        x = x+self.net(x)
        #print("After residial block:",x.shape)
        return x
#Upsample Block
class Upsample_Block(nn.Module):
    def __init__(self, in_ch,scale_factor):
        super(Upsample_Block, self).__init__()
        self.net  = nn.Sequential(
        nn.Conv2d(in_channels = in_ch, out_channels = in_ch*(scale_factor**2), kernel_size = 3, stride = 1, padding = 1),
        nn.PixelShuffle(upscale_factor = scale_factor),
        nn.PReLU()
        )
    def forward(self, x):
        x = self.net(x)
        #print("After upsampling:",x.shape)
        return x
        

#Generator Block
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        #First Block
        self.block1 = nn.Sequential(
        nn.Conv2d(in_channels = 3, out_channels = 64, kernel_size = 9, padding = 4,stride = 1),
        nn.PReLU()
        )
        #8 Residual Blocks
        self.residuals = nn.ModuleList([Residual_Block() for x in range(8)]) 
        self.block2 = nn.Sequential(
        nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size = 3, stride = 1,padding = 1),
        nn.BatchNorm2d(num_features = 64)
        )
        # 2 Upsampling Blocks
        self.upsamples = nn.Sequential(
        Upsample_Block(64,2),
        Upsample_Block(64,2),
        nn.Conv2d(in_channels = 64, out_channels = 3, kernel_size = 9, stride = 1, padding = 4)
        )
    def forward(self, x):
        #print("Initial shape:",x.shape)
        x = self.block1(x)
        #print("After Block 1:", x.shape)
        x_res = self.residuals[0](x)
        for i in range(7):
            x_res = self.residuals[i+1](x_res)
        x_res = self.block2(x)
        x = x+x_res
        x = self.upsamples(x)
        #print("Output Shape:",x.shape)
        return x

### Discriminator Network

In [6]:
class Discriminator(nn.Module):
    def __init__(self, l=0.2):
        super(Discriminator, self).__init__()
        self.net = nn.Sequential(
        nn.Conv2d(in_channels = 3, out_channels = 64, kernel_size = 3, stride = 1),
        nn.LeakyReLU(l),
        
        nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size = 3, stride = 2, padding = 1),
        nn.BatchNorm2d(num_features = 64),
        nn.LeakyReLU(l),
            
        nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size = 3, stride = 1, padding = 1),
        nn.BatchNorm2d(num_features = 128),
        nn.LeakyReLU(l),
        
        nn.Conv2d(in_channels = 128, out_channels = 128, kernel_size = 3, stride = 2, padding = 1),
        nn.BatchNorm2d(num_features = 128),
        nn.LeakyReLU(l),
            
        nn.Conv2d(in_channels = 128, out_channels = 256, kernel_size = 3, stride = 1, padding = 1),
        nn.BatchNorm2d(num_features = 256),
        nn.LeakyReLU(l),
            
        nn.Conv2d(in_channels = 256, out_channels = 256, kernel_size = 3, stride = 2, padding = 1),
        nn.BatchNorm2d(num_features = 256),
        nn.LeakyReLU(l),
            
        nn.Conv2d(in_channels = 256, out_channels = 512, kernel_size = 3, stride = 1, padding = 1),
        nn.BatchNorm2d(num_features = 512),
        nn.LeakyReLU(l),
            
        nn.Conv2d(in_channels = 512, out_channels = 512, kernel_size = 3, stride = 2, padding = 1),
        nn.BatchNorm2d(num_features = 512),
        nn.LeakyReLU(l),
            
        nn.AdaptiveAvgPool2d(1),
        nn.Conv2d(512, 1024, kernel_size = 1),
        nn.LeakyReLU(l),
        nn.Conv2d(1024,1,kernel_size = 1)
        )
    def forward(self,x):
        y = self.net(x)
        yhat = torch.sigmoid(y).view(x.shape[0])
        return yhat

Lets test the dimensional correctness with a random vector.

In [7]:
x = torch.randn((5,3,512,512))

disc = Discriminator()

y = disc(x)

y.data.shape

torch.Size([5])

Look Good!

##  Train

The generic method of training GANs is given bellow. It was proposed by Ian Goodfellow et al in the first paper that introduced GANs.

<img src = "https://3qeqpr26caki16dnhd19sv6by6v-wpengine.netdna-ssl.com/wp-content/uploads/2019/05/Summary-of-the-Generative-Adversarial-Network-Training-Algorithm-1024x669.png" width = 600>

In [8]:
HR_SIZE = 512
LR_SIZE = 128
SCALING_FACTOR = 4
PRETRAIN_EPOCHS =2
EPOCHS = 1

I will first write the basic backbone training function, then add other functionalities such as-
<ul>
    <li>Sending model to GPU
    <li>Label Smoothing</li>
    <li>Label Flipping</li>
    <li>Saving checkpoints when training</li>
    <li>Visualizing the model with Tensorboard</li>
    <li>Displaying epochwise performance</li>

In [14]:
netG = Generator()
netD = Discriminator()

optimizerD = torch.optim.Adam(netD.parameters())
optimizerG = torch.optim.Adam(netG.parameters())
mse = nn.MSELoss()
bce = nn.BCELoss()
pretrain = True
#criterionG = 
#criterionD = 


test_set
train_set
train_loader
test_loader

### Pretraining Generator

In [ ]:
def pre_train():
    optimizerG = torch.optim.Adam(netG.parameters())
    print("Pretraining Generator:")
    for epoch in range(PRETRAIN_EPOCHS):
        print("\tEpoch: ",epoch)
        netG.train()
        for lr, hr in tqdm(train_loader):
            netG.zero_grad()
            sr  = netG(lr)
            loss = mse(lr,hr)
            loss.backward()
            optimizerG.step()